In [26]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import seaborn as sns

# Baca data dari CSV
data = pd.read_csv("makanan9.csv")

# Drop kolom yang tidak diperlukan
data = data.drop(['id', 'kode', 'sumber', 'gambar', 'satuan'], axis=1)
data['jenis_pangan_encoded'] = pd.factorize(data['jenis_pangan'])[0] 

# Ubah nilai yang asalnya ',' menjadi '.'
numeric_cols = ['jenis_pangan_encoded', 'energi_kal', 'protein_gram', 'lemak_gram', 'karbohidrat_gram', 'serat_gram',
                'kalsium_mg', 'fosfor_mg', 'zatbesi_mg', 'natrium_mg', 'kalium_mg', 'vitc_mg']

data[numeric_cols] = data[numeric_cols].replace({',': '.'}, regex=True)
data[numeric_cols] = data[numeric_cols].astype(float)

# Normalisasi data
scaler = StandardScaler()
data_normalized = scaler.fit_transform(data[numeric_cols])

# Menggunakan hierarchical clustering
linked = linkage(data_normalized, method='ward')
data['cluster'] = fcluster(linked, t=10, criterion='maxclust')

# Hitung similarity matrix (Cosine Similarity)
cosine_sim = cosine_similarity(data_normalized, data_normalized)

# Fungsi untuk mendapatkan rekomendasi makanan
def get_recommendations(food_names, allergy_list):
    # Cari indeks makanan yang cocok dengan nama makanan yang diberikan
    food_indices = [data[data['nama_bahan'] == food_name].index[0] for food_name in food_names]
    
    print("Makanan tersebut berada di indeks ke-", food_indices)
    
    # Filter makanan berdasarkan alergi
    filtered_data = data.copy()
    for allergy in allergy_list:
        filtered_data = filtered_data[~filtered_data['nama_bahan'].str.contains(allergy, case=False)]
        filtered_data = filtered_data[~filtered_data['jenis_pangan'].str.contains(allergy, case=False)]
        
    # 500 dari buku KIA batas natrium untuk ibu hamil, 900 dari AKG batas untuk ibu hamil 
    filtered_data = filtered_data[(filtered_data['natrium_mg'] < 500) | (filtered_data['retinol_mcg'] < 900)]
    
    # Cluster dari makanan yang dimiliki user
    user_clusters = data.loc[food_indices, 'cluster'].unique()
    
    # Filter data berdasarkan cluster yang sama dengan makanan user
    cluster_data = filtered_data[filtered_data['cluster'].isin(user_clusters)]
    
    # Hitung similarity antara makanan yang dimiliki user dengan makanan yang tersedia dalam cluster yang sama
    sim_scores = []
    for food_index in food_indices:
        sim_scores.extend(list(enumerate(cosine_sim[food_index])))
    
    # Filter sim_scores berdasarkan makanan dalam cluster yang sama
    sim_scores = [(idx, score) for idx, score in sim_scores if idx in cluster_data.index]
    
    # Hitung rata-rata similarity score untuk setiap makanan yang tersedia
    sim_scores_df = pd.DataFrame(sim_scores, columns=['index', 'score'])
    avg_sim_scores = sim_scores_df.groupby('index')['score'].mean().reset_index()
    
    # Urutkan makanan berdasarkan similarity score
    avg_sim_scores = avg_sim_scores.sort_values(by='score', ascending=False)
    
    # Ambil 10 makanan dengan similarity score tertinggi (kecuali makanan yang dimiliki user)
    top_similar_food_indices = avg_sim_scores['index'].iloc[:10].tolist()  # Mengambil 10 teratas
    top_similar_food_names = data['nama_bahan'].iloc[top_similar_food_indices].tolist()
    top_similar_food_scores = avg_sim_scores['score'].iloc[:10].tolist()  # Nilai similarity
    
    return top_similar_food_names, top_similar_food_scores

# Contoh penggunaan
allergy_list = ['susu']  # Ganti dengan alergi yang dimiliki user
food_names = ['jeruk bali segar']  # Ganti dengan makanan yang dimiliki user
recommendations, similarity_scores = get_recommendations(food_names, allergy_list)
print("Rekomendasi makanan:")
for idx, (food, score) in enumerate(zip(recommendations, similarity_scores)):
    print(f"{idx+1}. {food} (Similarity Score: {score})")


Makanan tersebut berada di indeks ke- [1028]
Rekomendasi makanan:
1. jeruk bali segar (Similarity Score: 1.0)
2. jeruk manis segar (Similarity Score: 0.9927083836359574)
3. jeruk ragi segar (Similarity Score: 0.97617981680335)
4. jeruk garut-keprok (Similarity Score: 0.9743143048188613)
5. arbai segar (Similarity Score: 0.9153941867347947)
6. vigus segar (Similarity Score: 0.8757975080315159)
7. lemon segar (Similarity Score: 0.8754449139085781)
8. kiwi segar (Similarity Score: 0.8689253686990115)
9. Boros kunci, segar (Similarity Score: 0.8687779110069834)
10. labu waluh segar (Similarity Score: 0.8599602547717492)


In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

# Baca data dari CSV
data = pd.read_csv("makanan9.csv")

# Drop kolom yang tidak diperlukan
data = data.drop(['id', 'kode', 'sumber', 'gambar', 'satuan'], axis=1)
data['jenis_pangan_encoded'] = pd.factorize(data['jenis_pangan'])[0] 

# Ubah nilai yang asalnya ',' menjadi '.'
numeric_cols = ['air_gram', 'energi_kal', 'protein_gram', 'lemak_gram', 'karbohidrat_gram', 'serat_gram',
                'kalsium_mg', 'fosfor_mg', 'zatbesi_mg', 'natrium_mg', 'kalium_mg', 'tembaga_mg', 'vitc_mg', 'jenis_pangan_encoded']

data[numeric_cols] = data[numeric_cols].replace({',': '.'}, regex=True)
data[numeric_cols] = data[numeric_cols].astype(float)

# Normalisasi data
scaler = StandardScaler()
data_normalized = scaler.fit_transform(data[numeric_cols])

# Menggunakan Nearest Neighbors
model = NearestNeighbors(n_neighbors=10, metric='cosine')
model.fit(data_normalized)


# Fungsi untuk mendapatkan rekomendasi makanan
def get_recommendations(food_names, allergy_list):
    # Cari indeks makanan yang cocok dengan nama makanan yang diberikan
    food_indices = [data[data['nama_bahan'] == food_name].index[0] for food_name in food_names]
    
    print("Makanan tersebut berada di indeks ke-", food_indices)
    
    # Filter makanan berdasarkan alergi
    filtered_data = data.copy()
    for allergy in allergy_list:
        filtered_data = filtered_data[~filtered_data['nama_bahan'].str.contains(allergy, case=False)]
        filtered_data = filtered_data[~filtered_data['jenis_pangan'].str.contains(allergy, case=False)]
        
    # 500 dari buku KIA batas natrium untuk ibu hamil, 900 dari AKG batas untuk ibu hamil 
    filtered_data = filtered_data[(filtered_data['natrium_mg'] < 500) | (filtered_data['retinol_mcg'] < 900)]
    
    # Mencari tetangga terdekat
    distances, indices = model.kneighbors(data_normalized[food_indices])
    
    print(distances)
    print(indices)
    
    # Mendapatkan nama dan skor kemiripan makanan
    recommendations = []
    similarity_scores = []
    for idx_list, dist_list in zip(indices, distances):
        for idx, dist in zip(idx_list, dist_list):
            if idx not in food_indices:
                recommendations.append(data.iloc[idx]['nama_bahan'])
                similarity_scores.append(1 - dist)  # Cosine similarity is 1 - cosine distance
    
    return recommendations[:10], similarity_scores[:10]  # Mengambil 10 teratas

# Contoh penggunaan
allergy_list = ['susu']  # Ganti dengan alergi yang dimiliki user
food_names = ['Ikan tuna']  # Ganti dengan makanan yang dimiliki user
recommendations, similarity_scores = get_recommendations(food_names, allergy_list)
print("Rekomendasi makanan:")
for idx, (food, score) in enumerate(zip(recommendations, similarity_scores)):
    print(f"{idx+1}. {food} (Similarity Score: {score})")


Makanan tersebut berada di indeks ke- [4]
[[0.         0.04221338 0.04493153 0.04904019 0.05992768 0.06638899
  0.06693172 0.06970239 0.06977292 0.07433868]]
[[   4   35  481  482  117  632  448  583 1116  478]]
Rekomendasi makanan:
1. ikan pomo segar (Similarity Score: 0.9577866183660159)
2. ikan mujahir pepes (Similarity Score: 0.9550684732122308)
3. ikan ekor kuning segar (Similarity Score: 0.950959812281062)
4. cakalang asar (asap) (Similarity Score: 0.9400723235114283)
5. ikan lehoma segar (Similarity Score: 0.9336110120903375)
6. ikan bandeng segar (Similarity Score: 0.9330682758069165)
7. ikan cakalang jantung segar (Similarity Score: 0.9302976111017128)
8. ikan lais bakar (Similarity Score: 0.9302270801244537)
9. ikan tempahas segar (Similarity Score: 0.9256613169311981)
